In [1]:
import pandas as pd
import numpy as np
import wandb
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV

In [2]:
# Initialize Weights & Biases run for tracking
wandb.init(project="useful_comments_sorter", name="run_initial")

wandb: Currently logged in as: michelmortier1213 (michelmortier1213-hes-so-gen-ve) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
# Load comment data with embeddings from JSON file
df = pd.read_json("data/comments_with_embeddings.json", lines=False)

In [3]:
df = pd.json_normalize(df["comments"])
df.head()

,at,score,content,reviewCreatedVersion,relevance_score,vector
0,2024-02-21T20:52:42,3,Je suis sur facebook depuis 2013 he vous adore...,451.0.0.45.109,85,"[0.021481344476342, 0.07381305843591601, -0.00..."
1,2020-05-22T09:53:04,5,J'adore cette application sauf que pour se met...,6.10.0.541,50,"[0.013015237636864001, 0.08942589908838201, -0..."
2,2020-06-18T20:17:52,5,Il s'agit là d'une très bonne application et d...,135.0.0.28.119,75,"[0.019543664529919, 0.047836035490036004, -0.0..."
3,2021-03-09T07:28:29,5,"depuis la mise à jour, l'appli s'ouvre et se r...",22.5.0.100,85,"[-0.06765174865722601, -0.114827893674373, -0...."
4,2021-01-19T20:22:29,1,Très déçue du sav amazon j'attends un rembours...,16.02.0.100,0,"[0.13606958091259, 0.09106799960136401, -0.014..."


In [4]:
X = df["vector"].tolist()
X = np.array(X, dtype=np.float32)
X = np.array(X).astype(np.float32)
y = df["relevance_score"].astype(float)

In [5]:
y_binary = (y >= 80).astype(int)

In [6]:

# Split the data into train and test sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

In [7]:
n_pos = sum(y_train == 1)
n_neg = sum(y_train == 0)

# Compute class imbalance ratio for scale_pos_weight
ratio = n_neg / n_pos

In [17]:
clf = xgb.XGBClassifier(
    tree_method="hist",          # GPU-compatible method (used instead of 'gpu_hist')
    device="cuda",               # Enable GPU acceleration
    verbosity=1,                 # Show training progress
    eval_metric="logloss",       # Metric for binary classification
    use_label_encoder=False,     # Disable label encoder (deprecated behavior)
    scale_pos_weight=ratio,       # Handle class imbalance
)

# Hyperparameter grid for tuning
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [4, 6, 8],
    "learning_rate": [0.05, 0.1],
    "subsample": [0.8, 1.0],
}

# Grid search with cross-validation (parallelized over all CPU cores)
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring="f1",
    cv=3,
    verbose=2
)

In [132]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:58:37] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END learning_rate=0.05, max_depth=4, n_estimators=100, subsample=0.8; total time=   3.7s


/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:58:41] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END learning_rate=0.05, max_depth=4, n_estimators=100, subsample=0.8; total time=   3.3s


/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:58:44] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END learning_rate=0.05, max_depth=4, n_estimators=100, subsample=0.8; total time=   3.5s


/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:58:48] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


KeyboardInterrupt: 

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# Define the search space for Bayesian optimization
search_space = {
    "n_estimators": Integer(100, 300),
    "max_depth": Integer(3, 10),
    "learning_rate": Real(0.01, 0.2, prior="log-uniform"),
    "subsample": Real(0.5, 1.0),
    "colsample_bytree": Real(0.5, 1.0),
    "gamma": Real(0, 5),
    "min_child_weight": Integer(1, 10),
    "reg_alpha": Real(0, 2),     # L1 regularization
    "reg_lambda": Real(0, 2),    # L2 regularization
}

# Bayesian hyperparameter search
bayes_search = BayesSearchCV(
    estimator=clf,
    search_spaces=search_space,
    n_iter=30,
    scoring="f1",
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=1
)

bayes_search.fit(X_train, y_train)


Fitting 3 folds for each of 1 candidates, totalling 3 fits


/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [13:15:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END colsample_bytree=0.705051979426657, gamma=3.6386287158866253, learning_rate=0.16356457461011642, max_depth=5, min_child_weight=7, n_estimators=183, reg_alpha=0.701862669798288, reg_lambda=1.479008468380841, subsample=0.6522316555182531; total time=   8.6s


/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [13:15:32] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END colsample_bytree=0.705051979426657, gamma=3.6386287158866253, learning_rate=0.16356457461011642, max_depth=5, min_child_weight=7, n_estimators=183, reg_alpha=0.701862669798288, reg_lambda=1.479008468380841, subsample=0.6522316555182531; total time=   5.5s


...

[CV] END colsample_bytree=0.5, gamma=5.0, learning_rate=0.05123396574074597, max_depth=8, min_child_weight=1, n_estimators=300, reg_alpha=2.0, reg_lambda=2.0, subsample=0.5; total time=  19.3s


/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:08:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END colsample_bytree=0.5, gamma=5.0, learning_rate=0.05123396574074597, max_depth=8, min_child_weight=1, n_estimators=300, reg_alpha=2.0, reg_lambda=2.0, subsample=0.5; total time=  22.9s


/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:08:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


BayesSearchCV(cv=3,
              estimator=XGBClassifier(base_score=None, booster=None,
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None, device='cuda',
                                      early_stopping_rounds=None,
                                      enable_categorical=False,
                                      eval_metric='logloss', feature_types=None,
                                      feature_weights=None, gamma=None,
                                      grow_policy=None, importance_type=None,
                                      interaction_constra...
                             'min_child_weight': Integer(low=1, high=10, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=300, prior='uniform', transform='normalize'),
                             'reg_alpha': Real(low=0, high=2, prior='uniform', transform='normalize'),
                             'reg_lambda': Real(low=0, high=2, prior='uniform', transform='normalize'),
                             'subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')},
              verbose=2)

In [8]:
model_params = {
    'learning_rate': 0.1,
    'max_depth': 8,
    'n_estimators': 200,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

In [19]:
run = wandb.init(
    project="useful_comments_sorter",
    name="xgb_1_cross_export",
    config=model_params,       # Logged under the “Config” tab in W&B
    reinit='finish_previous'
          # Restart the run if a previous one was active
)

In [9]:
from wandb.integration.xgboost import WandbCallback
import xgboost as xgb

# # W&B callback for logging model and feature importance
# wb_cb = WandbCallback(
#     log_model=True,
#     log_feature_importance=True
# )

In [10]:
model = xgb.XGBClassifier(
    **model_params,
    tree_method="gpu_hist",       # Use GPU-accelerated histogram-based algorithm
    device="cuda",                # Run on GPU
    # callbacks=[wb_cb],            # Log metrics and feature importance to W&B
    scale_pos_weight=ratio        # Handle class imbalance
)

In [11]:
eval_set = [(X_train, y_train), (X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=True)

/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [17:42:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [17:42:22] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[0]	validation_0-logloss:0.64734	validation_1-logloss:0.65061
[1]	validation_0-logloss:0.60830	validation_1-logloss:0.61433
[2]	validation_0-logloss:0.57476	validation_1-logloss:0.58365
[3]	validation_0-logloss:0.54578	validation_1-logloss:0.55714
[4]	validation_0-logloss:0.52046	validation_1-logloss:0.53493
[5]	validation_0-logloss:0.49810	validation_1-logloss:0.51512
[6]	validation_0-logloss:0.47820	validation_1-logloss:0.49776
[7]	validation_0-logloss:0.46057	validation_1-logloss:0.48282
[8]	validation_0-logloss:0.44494	validation_1-logloss:0.46949
[9]	validation_0-logloss:0.43090	validation_1-logloss:0.45762
[10]	validation_0-logloss:0.41839	validation_1-logloss:0.44741
[11]	validation_0-logloss:0.40627	validation_1-logloss:0.43760
[12]	validation_0-logloss:0.39568	validation_1-logloss:0.42913
[13]	validation_0-logloss:0.38553	validation_1-logloss:0.42145
[14]	validation_0-logloss:0.37660	validation_1-logloss:0.41452
[15]	validation_0-logloss:0.36834	validation_1-logloss:0.40851
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [12]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score

y_pred = model.predict(X_test)
recall = recall_score(y_test, y_pred)
y_proba = model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class

/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/core.py:2676: UserWarning: [17:42:51] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
/home/marco/projects/flawfinder/useful-comments-sorter-model/.venv/lib/python3.12/site-packages/xgboost/core.py:729: UserWarning: [17:42:51] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once

In [ ]:
# wandb.log({
#     "confusion_matrix": wandb.plot.confusion_matrix(
#         preds=y_pred.tolist(),
#         y_true=y_test.tolist(),
#         class_names=["negative", "positive"]  # Adjust if class labels differ
#     ),
#     "f1": f1_score(y_test, y_pred),
#     "acc": accuracy_score(y_test, y_pred),
#     "auc": roc_auc_score(y_test, y_proba),
#     "recall": recall
# })

# run.finish()


acc,▁
auc,▁
epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇████
f1,▁
recall,▁
validation_0-logloss,█▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_1-logloss,█▅▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.85836
auc,0.93572
epoch,199
f1,0.8713


In [ ]:
import joblib

joblib.dump(model, "models/useful-comments-sorter_model_xgb1.joblib")

['models/useful-comments-sorter_model_.joblib']